In [2]:
import os
import glob

import numpy as np
from shutil import rmtree

def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)

def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)

def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())

clean()

In [8]:
import ioh
import numpy as np
%pip show ioh

Name: ioh
Version: 0.3.17
Summary: The experimenter for Iterative Optimization Heuristics
Home-page: https://iohprofiler.github.io/IOHexperimenter
Author: Jacob de Nobel, Furong Ye, Diederick Vermetten, Hao Wang, Carola Doerr and Thomas Bäck
Author-email: iohprofiler@liacs.leidenuniv.nl
License: BSD
Location: /Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Problem
In ioh, everything revolves around the `problem` class, which exists for both `Real` and `Integer` types for continious and discrete problems resp. These classes are wrappers around an objective function, and can be used to interact with the various other parts of iohexperimenter. 

We have a number of objective functions already implemented for convenience, which includes the objective functions from the BBOB single objective benchmark by the COCO platform for the continous case and the PBO benchmark functions for the discrete case.


In [2]:
# A list of problems can be accessed via the base classes
ioh.problem.RealSingleObjective.problems

{1: 'Sphere',
 2: 'Ellipsoid',
 3: 'Rastrigin',
 4: 'BuecheRastrigin',
 5: 'LinearSlope',
 6: 'AttractiveSector',
 7: 'StepEllipsoid',
 8: 'Rosenbrock',
 9: 'RosenbrockRotated',
 10: 'EllipsoidRotated',
 11: 'Discus',
 12: 'BentCigar',
 13: 'SharpRidge',
 14: 'DifferentPowers',
 15: 'RastriginRotated',
 16: 'Weierstrass',
 17: 'Schaffers10',
 18: 'Schaffers1000',
 19: 'GriewankRosenbrock',
 20: 'Schwefel',
 21: 'Gallagher101',
 22: 'Gallagher21',
 23: 'Katsuura',
 24: 'LunacekBiRastrigin',
 30: 'UniformStarDiscrepancy10',
 31: 'UniformStarDiscrepancy25',
 32: 'UniformStarDiscrepancy50',
 33: 'UniformStarDiscrepancy100',
 34: 'UniformStarDiscrepancy150',
 35: 'UniformStarDiscrepancy200',
 36: 'UniformStarDiscrepancy250',
 37: 'UniformStarDiscrepancy500',
 38: 'UniformStarDiscrepancy750',
 39: 'UniformStarDiscrepancy1000',
 40: 'SobolStarDiscrepancy10',
 41: 'SobolStarDiscrepancy25',
 42: 'SobolStarDiscrepancy50',
 43: 'SobolStarDiscrepancy100',
 44: 'SobolStarDiscrepancy150',
 45: 'Sobo

In [3]:
# In order to instantiate a problem instance, we can do the following:
problem = ioh.get_problem(
    "Sphere", 
    instance=1,
    dimension=10,
    problem_class=ioh.ProblemClass.REAL
)
problem

<RealSingleObjectiveProblem 1. Sphere (iid=1 dim=10)>

In [4]:
# The problem class includes information about the problem, which can be retrieved via the meta_data accessor
problem.meta_data.name

'Sphere'

In [5]:
# The current state of the problem, e.g. the number of evaluations, best seen points etc. are stored in the problems state.
problem.state

<State evaluations: 0 final_target_found: false current_best: <Solution x: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] y: inf>>

In [6]:
# Every problem as has a simple box-bounds associcated
problem.bounds

<BoxConstraint lb: [[-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]] ub: [[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]>

In [9]:
# We can access the contraint information of the problem
x0 = np.random.uniform(problem.bounds.lb, problem.bounds.ub)

# Evaluation happens like a 'normal' objective function would
problem(x0)

# Whenever the problem is evaluated, the state changes
problem.state

<State evaluations: 1 final_target_found: false current_best: <Solution x: [-0.795576614946512, -3.6522005733399543, 3.2319750993538534, 2.79489283035523, -1.125036689788094, -3.865696567433533, -2.868521744968977, -0.293342933208141, 3.3178854050844446, 3.721882056828216] y: 202.33697135568227>>

In [10]:
# Additionally, it is possible to evaluate a list of points
n_points = 5
X = np.random.uniform(problem.bounds.lb, problem.bounds.ub, size=(n_points, problem.meta_data.n_variables))
problem(X)

[217.45120408466346,
 169.63034944395196,
 130.45556882708766,
 288.13813252754255,
 230.07698095188476]

In [12]:
# If we want to perform multiple runs with the same objective function, after every run, the problem has to be reset, 
# such that the internal state reflects the current run.
def run_experiment(problem, algorithm, n_runs=5):
    for run in range(n_runs):
        
        # Run the algorithm on the problem
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found:{problem.state.current_best.y: .3f}")

        # Reset the problem
        problem.reset()

In [13]:
class RandomSearch:
    'Simple random search algorithm'
    def __init__(self, n: int, length: float = 0.0):
        self.n: int = n
        self.length: float = length
        
    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.uniform(problem.bounds.lb, problem.bounds.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)             

In [14]:
# using the random search algorithm, we can then run a simple experiment
run_experiment(problem, RandomSearch(10))

run: 1 - best found: 117.046
run: 2 - best found: 128.557
run: 3 - best found: 126.402
run: 4 - best found: 136.491
run: 5 - best found: 125.278


# Adding custom problems
As the list of already implemented problems might not contains the objective function you would like to analyze and benchmark, we include an easy to use interface to wrap your benchmark function into an iohexperimenter `problem` object. 

Currently, only single objective functions are supported, so the only requirement on the function is its signature, which should take a single array parameter, and return a single floating point number $f(\mathbf{x}) \mapsto \mathbb{R}$.

In this example we will add the function (Styblinski–Tang):

$f(\mathbf{x}) = \frac{\sum_{i=1}^n x_i^4 - 16x_i^2 + 5x_i}{2}$

defined on $[-5, 5]$

In [15]:
def styblinski_tang(x: np.ndarray) -> float:
    return np.sum(np.power(x, 4) - (16 * np.power(x, 2)) + (5 * x)) / 2


styblinski_tang(np.array([-2.903534]*10)) # global minima

-391.661657037714

In [16]:
# we can wrap this function in ioh, as this is is a continous function, we use wrap_real_problem:
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     # Handle to the function
    name="StyblinskiTang",                               # Name to be used when instantiating
    optimization_type=ioh.OptimizationType.MIN, # Specify that we want to minimize
    lb=-5,                                               # The lower bound
    ub=5,                                                # The upper bound
)

In [17]:
# We can create an instance of this problem, wrapped in ioh
problem = ioh.get_problem("StyblinskiTang", dimension=10)
problem

<RealSingleObjectiveProblem 1121. StyblinskiTang (iid=1 dim=10)>

In [18]:
problem(np.array([-2.903534]*10))

-391.661657037714

## Specifying instances
When doing bechmarking of custom problems, it can often be usefull to look at different instances of the same problem, but that perform some transformation the parameter or objective space, to test if an algorithm is invariant to such transformations. 

Here we add the following transformations to the aforementioned objective function:

$T_x(\mathbf{x}, c) = \mathbf{x} + c$

$T_y(y, c) = y * c$

We can do this by providing transformation functions to the `wrap_problem` interface. Note that these take two parameters:
 1. The operand they function on, which are the variables for the variables transformation and objective function value for the objective transformation. 
 2. An integer identifier of the instance id which is currently used. This allows you to specify alternate behavior for different instances.  

In [19]:
# Variables transformation R^d -> R^d
def transform_variables(x: np.ndarray, instance_id:int) -> np.ndarray:
    c = (instance_id - 1) * .5
    return x + c

# Objective transformation R -> R
def transform_objectives(y: float, instance_id:int) -> float:
    c = instance_id
    return y * c

# Note that we can overwrite a previously defined problem by calling wrap_real_problem again with the same name
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.MIN, 
    lb=-5,                                               
    ub=5,      
    
    # Adding the transformation functions
    transform_variables=transform_variables,     
    transform_objectives=transform_objectives
)

In [20]:
# We can now create different instances of the same problem
instance1 = ioh.get_problem("StyblinskiTang", instance=1, dimension=10)
instance2 = ioh.get_problem("StyblinskiTang", instance=2, dimension=10)
instance1, instance2

(<RealSingleObjectiveProblem 1121. StyblinskiTang (iid=1 dim=10)>,
 <RealSingleObjectiveProblem 1121. StyblinskiTang (iid=2 dim=10)>)

In [21]:
# Note that when evaluating with the same point, each instance gives a different (transformed value)
instance1(x0), instance2(x0)

(-174.66495533498738, -225.2422460074127)

# Logging data
The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer. 

In [22]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /Users/jiaxuanyu/Code/EA/A1/my-experiment>

In [23]:
ls()

zsh: parse error near `()'


In [24]:
# In order to log data for a problem, we only have to attach it to a logger
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)

# We can then run the random search as before, only now all data will be logged to a file
run_experiment(problem, RandomSearch(10), n_runs=1)

# Force logger to flush and write files. (> v0.3.3)
# This is only required when using the logger in an interactive context, such as this jupyter notebook.
# Otherwise, the destructor, which is called on exit of the python interpreter, forces such a write automatically.
logger.close()

run: 1 - best found:-68.015


In [25]:
StyblinskiTang_function_identifier = max(ioh.problem.RealSingleObjective.problems.keys())
StyblinskiTang_function_identifier

1121

In [ ]:
cat(f"my-experiment/IOHprofiler_f{StyblinskiTang_function_identifier}_StyblinskiTang.json")

In [ ]:
cat(f"my-experiment/data_f{StyblinskiTang_function_identifier}_StyblinskiTang/IOHprofiler_f{StyblinskiTang_function_identifier}_DIM2.dat")

## Triggers 
The default behavior of the `Analyzer` logger is to log data only when there is an improvement of the objective value. We can change this behaviour, by specifying one or more triggers, which are logical operators, which when one of them evaluates to True, will cause data to be logged.

We provide a number of trigger variants which can be use to customize the logging. In the following example, a trigger is defined which evaluates to True, every 3 function evaluations. It is combined with a trigger for improvement, so data will be logged on every 3rd function evaluation, or when there is an observed improvement of the objective value

In [26]:
triggers = [
    ioh.logger.trigger.Each(3),
    ioh.logger.trigger.OnImprovement()
]

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # Add the triggers to the logger
    triggers = triggers
)

logger

<Analyzer /Users/jiaxuanyu/Code/EA/A1/my-experiment-1>

In [27]:
# Rerun the same experiment as before
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)
logger.close()

run: 1 - best found:-44.260


In [ ]:
# We can now see that data is logged either if there is improvement, or on every 3rd evaluation
cat(f"my-experiment-1/data_f{StyblinskiTang_function_identifier}_StyblinskiTang/IOHprofiler_f{StyblinskiTang_function_identifier}_DIM2.dat")

## Properties
If we want to keep track of any dynamic parameters a given algorithm might have, we can use properties to log them to the output files. 

In the following example, we will track the length parameters for the RandomSearch algorithm, which is added for illustrative purpoposes, and changes for every function evaluation

In [28]:
# RandomSearch has a length parameter, which is dynamic
algorithm = RandomSearch(10)

# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True
)

# Before we attach a problem, we tell the logger to keep track of the length parameter on algorithm
logger.watch(algorithm, "length")

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, algorithm, n_runs=1)
logger.close()

run: 1 - best found:-47.829


In [ ]:
# Note the additional length parameter being logged 
cat(f"my-experiment-2/data_f{StyblinskiTang_function_identifier}_StyblinskiTang/IOHprofiler_f{StyblinskiTang_function_identifier}_DIM2.dat")

# Alternate logging behaviour
We provide a number of different loggers in addition to the `Analyzer` logger, which include:
 - `FlatFile` which logs data to a simple csv file
 - `Store` which keeps all of the stored data in memory
 - `EAF/EAH` which compute Empirical Attainment Function/Histogram statistics on the fly

You can define your own custom logging behavoir by inheriting from the `AbstractLogger` class. The only required part of the interface is that you override the `__call__` operator, which takes a single `ioh.LogInfo` parameter. In this method you should define your desired behavior. 

In [ ]:
# Simple logger that logs data using the python logging module whenever it is triggeredd
class MyLogger(ioh.logger.AbstractLogger):
    def __call__(self, log_info: ioh.LogInfo):
        print(f"triggered! y: {log_info.y}")


# The abstract logger takes two parameters, triggers and properties
mylogger = MyLogger(triggers=[ioh.logger.trigger.ALWAYS])

problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(mylogger)

run_experiment(problem, RandomSearch(10), 1)

# Standardized Experimental Setup (Python only)
In Python, we provide the `Experiment` class which can be used to easily run a given algorithm over a larger number of problems. 

In [ ]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = [1, StyblinskiTang_function_identifier],               # the id's of the problems we want to test
    iids = [1, 10],               # the instances 
    dims = [2, 10],               # the dimensions
    reps = 3,                     # the number of runs,
    zip_output = True,
    old_logger = True       
)

In [ ]:
experiment.run()

In [ ]:
ls("ioh_data")

In [ ]:
clean()

In [ ]:
# cleanup
rmtree("my-experiment", ignore_errors=True)
rmtree("ioh_data", ignore_errors=True)

# Working with constraints (> v0.3.3)
Every problem has a set of constraints associated with it. For most of the currently implemented problems, this is an empty set by default. However, we can modify this set, and add arbitrary functions as constraints. 

In [29]:
# We take the Shere function as example
p = ioh.get_problem("Sphere", 1, 2)

# The set of constraints is empty by default
p.constraints


<ConstraintSet: >

In [30]:
# We can turn the bounds of the problem into a contraint
print(p.bounds)

# By default, bounds are not enforced, so violating them changes nothing to the returned objective function value
p([10, 10])

<BoxConstraint lb: [[-5, -5]] ub: [[5, 5]]>


298.96209408

In [31]:
# For bound constraints, violation is computed as the sum of the squared violation per component
p.enforce_bounds(
    how=ioh.ConstraintEnforcement.SOFT, # The enforcement strategy: SOFT ensure penalization on violation by a penalty (y + p)
    weight=1.0,                         # Penaly p is computed as: weight * violation ^ exponent                       
    exponent=1.0,                        
)
# Now the bounds are in the constraint set
p.constraints

<ConstraintSet: <BoxConstraint lb: [[-5, -5]] ub: [[5, 5]]>>

In [32]:
# We can now observe the added penalty term
p([10, 10]) 

348.96209408

In [ ]:
# There a several strategies that modify a constraint's behavoir:
types = (
    ioh.ConstraintEnforcement.NOT,       # Don't calulate the constraint function
    ioh.ConstraintEnforcement.HIDDEN,    # Calulate the constraint, but don't penalize
    ioh.ConstraintEnforcement.SOFT,      # Calculate both constraint and objective function value y, and penalize as y + p
    ioh.ConstraintEnforcement.HARD,      # Calulate the constraint, if there is violation, don't calculate y, and only return p
)
for strategy in types:
    p.enforce_bounds(how=strategy, weight=1.0, exponent=1.0)
    print(strategy, p([10, 10]), p.constraints.violation())


In [ ]:
# Note that when we use a HARD method, the objective value is actually lower (50.0) than the actual objective value (298.96)
# This is because we didn't change any of the other parameters, that influence the penalty. Normally, with HARD constraints, we 
# want to also make sure that we return a y-value that is infeasible. We can do this by changing the weight and/or exponent parameters.
p.enforce_bounds(
    how=ioh.ConstraintEnforcement.HARD, 
    weight=float("inf"),                
    exponent=1.0                        
)

# Any infeasible solution gets an infinity value
p([10, 10])

In [ ]:
# Note that the actual objective function is not called for infeasible points with HARD constraints:
p.state.y_unconstrained

In [ ]:
# If we pass a feasible point it is evaluated
p([1.,-1.]), p.state.y_unconstrained

### Custom constraint functions

In [ ]:
# We can create custom constraint functions, which can be 'just' functions that check something on x.
# Such functions should return a nonzero floating point value when there is violation, and zero otherwise. 

def is_x_ordered(x: np.ndarray) -> float:
    '''Checks that xi < xi+1'''

    return 1. - (np.diff(x) > 0).all().astype(float)

# x is not ordered, so there is a violation (1.)
print(is_x_ordered([10, 1, 3]))

# x is ordered, so there is no violation (0.)
print(is_x_ordered([1, 3, 10]))

In [ ]:
# We can add such a function to an arbitary problem as constraint like so:
constraint = ioh.RealConstraint(is_x_ordered, name="is_x_ordered", weight=100_000)

p = ioh.get_problem("Sphere", 1, 3)
p.add_constraint(constraint)

# Note that it gets added to the list of constraints. 
p.constraints 

In [ ]:
print(p([4, 1, 3]))  # The penalty gets added 
print(p([1, 3, 4]))  # The penalty doesn't get added

In [ ]:
# We can remove the contraint again
p.remove_constraint(constraint)
p.constraints

### Wrapping constraints

In [ ]:
# We can also add constraints to the problem definition to wrap problem
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.MIN, 
    lb=-5,                                              
    ub=5,   
    # Adding the constraints                                            
    constraints=[constraint]
)

In [ ]:
# Now if we instantiate an instance of the Styblinski-Tang function, is has the x_ordering constraint added automatically
p = ioh.get_problem("StyblinskiTang", 1, 5)
p.constraints

### Logging constraints

In [ ]:
# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # We add the constraint properties to the list of logged items
    # Note that these properties are aggregated values for ALL applied constrains, 
    # So when you have more than one constraint added to a problem, these values will be the sums
    # of all the applied constraints. 
    additional_properties=[
        ioh.logger.property.CURRENTY,   # The constrained y-value, by default only the untransformed & unconstraint y
                                        # value is logged. 
        ioh.logger.property.VIOLATION,  # The violation value
        ioh.logger.property.PENALTY     # The applied penalty
    ]
)

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)
logger.close()

In [ ]:
cat(f"my-experiment/IOHprofiler_f{StyblinskiTang_function_identifier}_StyblinskiTang.json")
cat(f"my-experiment/data_f{StyblinskiTang_function_identifier}_StyblinskiTang/IOHprofiler_f{StyblinskiTang_function_identifier}_DIM2.dat")

In [ ]:
clean()